In [5]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Get the parent directory of the current file.
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
# Add the parent directory to the Python path.
sys.path.append(parent_dir)

from ML.preprocessing import create_datasets
from ML.model import make_model

test_batches, train_batches, val_batches = create_datasets("/home/arbersejdiji/inria1358/patches/", 
                                                           batch_size=16, 
                                                           data_size=0.05)

✅ Test_batches with 11250 images (704 batches) created
✅ Train_batches with 1518 images (95 batches) created
✅ Val_batches with 3544 images (222 batches) created


In [6]:
#binary cross_entropy
model_ter, history_ter = make_model(train_batches, 
           val_batches,
           (200,200,3), 
           model_name='ternausnet')

model_seg, history_seg = make_model(train_batches, 
           val_batches,
           (200,200,3), 
           model_name='segnet')

✅ TernausNet initialized
✅ Model compiled
Epoch 1/2
95/95 [==============================] - 1336s 14s/step - loss: 0.4524 - accuracy: 0.8253 - mean_io_u: 0.4196 - val_loss: 0.3836 - val_accuracy: 0.8431 - val_mean_io_u: 0.4190
Epoch 2/2
95/95 [==============================] - 1243s 13s/step - loss: 0.3398 - accuracy: 0.8486 - mean_io_u: 0.4196 - val_loss: 0.3741 - val_accuracy: 0.8382 - val_mean_io_u: 0.4190
✅ Model trained on 95 batches over 2 epochs.
✅ SEGNET Model initialized
✅ Model compiled
Epoch 1/2
95/95 [==============================] - 853s 9s/step - loss: 0.3319 - accuracy: 0.8640 - mean_io_u_1: 0.4196 - val_loss: 0.4104 - val_accuracy: 0.8380 - val_mean_io_u_1: 0.4190
Epoch 2/2
95/95 [==============================] - 835s 9s/step - loss: 0.2972 - accuracy: 0.8777 - mean_io_u_1: 0.4196 - val_loss: 0.4778 - val_accuracy: 0.8380 - val_mean_io_u_1: 0.4190
✅ Model trained on 95 batches over 2 epochs.


In [9]:
import tensorflow as tf

class DiceLoss(tf.keras.losses.Loss):
    def __init__(self, smooth=1e-6, gama=2):
        super(DiceLoss, self).__init__()
        self.name = 'NDL'
        self.smooth = smooth
        self.gama = gama

    def call(self, y_true, y_pred):
        y_true, y_pred = tf.cast(
            y_true, dtype=tf.float32), tf.cast(y_pred, tf.float32)
        nominator = 2 * \
            tf.reduce_sum(tf.multiply(y_pred, y_true)) + self.smooth
        denominator = tf.reduce_sum(
            y_pred ** self.gama) + tf.reduce_sum(y_true ** self.gama) + self.smooth
        result = 1 - tf.divide(nominator, denominator)
        return result

loss_function = DiceLoss(smooth=1e-6, gama=2)

In [11]:
#Dice Loss
model_dt, history_dt = make_model(train_batches, 
           val_batches,
           (200,200,3), 
           model_name='ternausnet',
           loss=loss_function)

model_ds, history_ds = make_model(train_batches, 
           val_batches,
           (200,200,3), 
           model_name='segnet',
           loss=loss_function)

✅ TernausNet initialized
✅ Model compiled
Epoch 1/2
95/95 [==============================] - 1143s 12s/step - loss: 0.4854 - accuracy: 0.7410 - mean_io_u_4: 0.4208 - val_loss: 0.3729 - val_accuracy: 0.8541 - val_mean_io_u_4: 0.4190
Epoch 2/2
95/95 [==============================] - 1155s 12s/step - loss: 0.3492 - accuracy: 0.8630 - mean_io_u_4: 0.4202 - val_loss: 0.3244 - val_accuracy: 0.8467 - val_mean_io_u_4: 0.4284
✅ Model trained on 95 batches over 2 epochs.
✅ SEGNET Model initialized
✅ Model compiled
Epoch 1/2
95/95 [==============================] - 779s 8s/step - loss: 0.3996 - accuracy: 0.8372 - mean_io_u_5: 0.4770 - val_loss: 0.8035 - val_accuracy: 0.8380 - val_mean_io_u_5: 0.4190
Epoch 2/2
95/95 [==============================] - 778s 8s/step - loss: 0.3431 - accuracy: 0.8716 - mean_io_u_5: 0.4756 - val_loss: 0.7988 - val_accuracy: 0.8380 - val_mean_io_u_5: 0.4190
✅ Model trained on 95 batches over 2 epochs.


In [ ]:
model_ds.predict()